In [1]:
from ishneholterlib import Holter
from pytz import timezone
import neurokit2 as nk
import yaml
import os
import zipfile
from datetime import datetime
from ecg_preproc import get_matching_files, extract_ecg_zip, get_holter_time_metadata

In [2]:
config = yaml.safe_load(open("config.yaml"))

In [4]:
pid = "9552"
zip_dir = os.path.join("data/raw_zip", pid)
zip_filepath = get_matching_files(zip_dir, "*ECG_ISHNE.zip")[0]
ecg_dir = os.path.join("data/raw_ecg", pid)
# extract_ecg_zip(zip_filepath, extract_dir=ecg_dir)  


In [5]:
ecg_filepath = get_matching_files(ecg_dir, "*.ecg")[4]
ecg_holter = Holter(ecg_filepath)
ecg_holter.load_data()
start_ts = datetime(ecg_holter.record_date.year,
                    ecg_holter.record_date.month,
                    ecg_holter.record_date.day,
                    ecg_holter.start_time.hour,
                    ecg_holter.start_time.minute,
                    ecg_holter.start_time.second)

In [20]:
dir(ecg_holter)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'autofill_header',
 'beat_anns',
 'birth_date',
 'checksum',
 'compute_checksum',
 'copyright',
 'deidentify',
 'ecg_block_offset',
 'ecg_size',
 'file_date',
 'file_version',
 'filename',
 'first_name',
 'get_header_bytes',
 'get_length',
 'id',
 'is_annfile',
 'is_valid',
 'last_name',
 'lead',
 'load_ann',
 'load_data',
 'load_header',
 'magic_number',
 'nleads',
 'pm',
 'proprietary',
 'race',
 'record_date',
 'recorder_type',
 'reserved',
 'sex',
 'sr',
 'start_time',
 'var_block',
 'var_block_offset',
 'var_block_size',
 'write_file']

In [24]:
ecg_holter.get_length()

datetime.timedelta(seconds=3600)

In [3]:
get_holter_time_metadata("data/raw_ecg/9552")

[{'dir': 'data/raw_ecg/9552',
  'filename': 'ECGRec_202128_C866077_2022022407.ecg',
  'start_time': datetime.datetime(2022, 2, 24, 15, 0),
  'end_time': datetime.datetime(2022, 2, 24, 16, 0)},
 {'dir': 'data/raw_ecg/9552',
  'filename': 'ECGRec_202128_C866077_2022022413.ecg',
  'start_time': datetime.datetime(2022, 2, 24, 21, 0),
  'end_time': datetime.datetime(2022, 2, 24, 22, 0)},
 {'dir': 'data/raw_ecg/9552',
  'filename': 'ECGRec_202128_C866077_2022022605.ecg',
  'start_time': datetime.datetime(2022, 2, 26, 13, 0),
  'end_time': datetime.datetime(2022, 2, 26, 14, 0)},
 {'dir': 'data/raw_ecg/9552',
  'filename': 'ECGRec_202128_C866077_2022022611.ecg',
  'start_time': datetime.datetime(2022, 2, 26, 19, 0),
  'end_time': datetime.datetime(2022, 2, 26, 20, 0)},
 {'dir': 'data/raw_ecg/9552',
  'filename': 'ECGRec_202128_C866077_2022022822.ecg',
  'start_time': datetime.datetime(2022, 3, 1, 6, 0),
  'end_time': datetime.datetime(2022, 3, 1, 7, 0)},
 {'dir': 'data/raw_ecg/9552',
  'filena

In [11]:
ecg = ecg_holter.lead[0].data
ecg_clean = nk.ecg_clean(ecg, sampling_rate=128)
x = nk.ecg_delineate(ecg_clean, sampling_rate=128)
# returns 2-tuple with dataframe and dictionary with location of events

In [18]:
x[1].keys()

dict_keys(['ECG_P_Peaks', 'ECG_P_Onsets', 'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets', 'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets'])

In [21]:
import numpy as np
np.arange(0, 3600, 60)





array([   0,   60,  120,  180,  240,  300,  360,  420,  480,  540,  600,
        660,  720,  780,  840,  900,  960, 1020, 1080, 1140, 1200, 1260,
       1320, 1380, 1440, 1500, 1560, 1620, 1680, 1740, 1800, 1860, 1920,
       1980, 2040, 2100, 2160, 2220, 2280, 2340, 2400, 2460, 2520, 2580,
       2640, 2700, 2760, 2820, 2880, 2940, 3000, 3060, 3120, 3180, 3240,
       3300, 3360, 3420, 3480, 3540])

In [1]:
from ecg_preproc import ECGProcessor, get_holter_end_time, get_matching_files, get_holter_start_time
from ishneholterlib import Holter
import neurokit2 as nk

pid = "9552"
ecg_dir = "data/raw_ecg/" + pid
ecg_filepath = get_matching_files(ecg_dir, "*.ecg")[34]
ecg_holter = Holter(ecg_filepath)
ecg_holter.load_data()
start_time = get_holter_start_time(ecg_holter)
end_time = get_holter_end_time(ecg_holter)
print(f"start_time: {start_time}")
print(f"end_time: {end_time}")

ecg = ecg_holter.lead[0].data
print(f"ecg len: {len(ecg)}")
ecg_processor = ECGProcessor(pid=pid, ecg_signal=ecg, start_time=start_time, end_time=end_time)
# len(ecg_processor.ecg_raw)
# print(ecg_processor.ecg_raw)
# ecg_clean = ecg_processor.clean(ecg_processor.ecg_raw)
# df_rpeaks, rpeaks = nk.ecg_peaks(ecg_clean, sampling_rate=128, method='rodrigues2021')
# print(ecg_processor.ecg_clean)
ecg_processor.run(".")


start_time: 2022-03-04 18:00:00
end_time: 2022-03-04 18:49:59
ecg len: 383872
[-0.054 -0.049 -0.055 ...  8.678  8.678  8.678]


In [8]:
import pandas as pd

df_signal = pd.read_parquet("ECG_signal_9552_2022-02-22 09:00:00_2022-02-22 10:00:00.parquet")
df_signal

# df_rpeaks = pd.read_parquet("ECG_rpeaks_9552_2022-02-22 09:00:00_2022-02-22 10:00:00.parquet")
# df_rpeaks

# df_phase = pd.read_parquet("ECG_phase_9552_2022-02-22 09:00:00_2022-02-22 10:00:00.parquet")
# df_phase

# df_qrs = pd.read_parquet("ECG_qrs_9552_2022-02-22 09:00:00_2022-02-22 10:00:00.parquet")
# df_qrs

,timestamp,ecg,quality,epoch
0,2022-02-22 09:00:00.000000000,0.000771,2,0
1,2022-02-22 09:00:00.007812500,-0.006416,2,0
2,2022-02-22 09:00:00.015625000,-0.018603,2,0
3,2022-02-22 09:00:00.023437500,-0.038041,2,0
4,2022-02-22 09:00:00.031250000,-0.063729,2,0
...,...,...,...,...
460795,2022-02-22 09:59:59.960937500,-0.021890,2,119
460796,2022-02-22 09:59:59.968750000,-0.022579,2,119
460797,2022-02-22 09:59:59.976562500,-0.023587,2,119
460798,2022-02-22 09:59:59.984375000,-0.024165,2,119


# run all for 9552

In [1]:
from ecg_preproc import ECGProcessor, get_holter_end_time, get_matching_files, get_holter_start_time
from ishneholterlib import Holter

pid = "9552"
ecg_dir = "data/raw_ecg/" + pid
ecg_filepaths = get_matching_files(ecg_dir, "*.ecg")

error_idx = []

for i, ecg_filepath in enumerate(ecg_filepaths):
    # if i <= 33:
    #     continue
    
    box_width = 50
    box_separator = "-" * box_width
    timestamp_format = "%Y-%m-%d %H:%M:%S"

    # Start of the box
    print(f"|{box_separator}|")

    # Processing information
    processing_info = f"processing PID {pid} file {i+1}/{len(ecg_filepaths)}"
    print(f"|{processing_info:<{box_width}}|")
    
    try:
        # Load Holter data
        ecg_holter = Holter(ecg_filepath)
        ecg_holter.load_data()

        # Start and end time
        start_time = get_holter_start_time(ecg_holter)
        end_time = get_holter_end_time(ecg_holter)
        print(f"|{'start_time:':<12}{start_time.strftime(timestamp_format):<{box_width-12}}|")
        print(f"|{'end_time:':<12}{end_time.strftime(timestamp_format):<{box_width-12}}|")

        # ECG length
        ecg = ecg_holter.lead[0].data
        print(f"|{'ecg len:':<12}{len(ecg):<{box_width-12}}|")

        # Processing ECG
        ecg_processor = ECGProcessor(pid=pid, ecg_signal=ecg, start_time=start_time, end_time=end_time)
        ecg_processor.run("data/proc_ecg/9552")
    except Exception as e:
        print(f"|{'Error:':<12}{str(e):<{box_width-12}}|")
        error_idx.append(i)
    # End of the box
    print(f"|{box_separator}|")

print(f"{len(ecg_filepaths)} files processed with {len(error_idx)} errors: {error_idx}")

|--------------------------------------------------|
|processing PID 9552 file 1/287                    |
|start_time: 2022-02-24 15:00:00                   |
|end_time:   2022-02-24 16:00:00                   |
|ecg len:    460800                                |
|--------------------------------------------------|
|--------------------------------------------------|
|processing PID 9552 file 2/287                    |
|start_time: 2022-02-24 21:00:00                   |
|end_time:   2022-02-24 22:00:00                   |
|ecg len:    460800                                |
|--------------------------------------------------|
|--------------------------------------------------|
|processing PID 9552 file 3/287                    |
|start_time: 2022-02-26 13:00:00                   |
|end_time:   2022-02-26 14:00:00                   |
|ecg len:    460800                                |
|--------------------------------------------------|
|---------------------------------------------

KeyboardInterrupt: 

# Plot participant daily cycles


In [2]:
pid = "9552"

# get all files from data/proc_ecg/{pid} that match "ECG_{pid}_meta_*.parquet"
meta_filepaths = get_matching_files(f"data/proc_ecg/{pid}", f"ECG_{pid}_meta_*.parquet")



['data/proc_ecg/9552/ECG_9552_meta_20220226_120000_20220226_130000_0.52_0.48_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220224_150000_20220224_160000_0.44_0.55_0.01.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220220_110000_20220220_120000_1.0_0.0_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220224_190000_20220224_200000_0.66_0.34_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220219_120000_20220219_130000_0.99_0.01_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220304_130000_20220304_140000_0.44_0.55_0.01.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220226_200000_20220226_210000_0.92_0.08_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220228_160000_20220228_170000_0.33_0.67_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220222_090000_20220222_100000_0.96_0.04_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220224_140000_20220224_150000_0.52_0.48_0.0.parquet',
 'data/proc_ecg/9552/ECG_9552_meta_20220224_210000_20220224_220000_0.81_0.19_0.0.parquet',